<div class="alert alert-success">  
    <h1 align="center" style="color:darkcyan;">Tabular Playground Series - Jun 2022</h1>  
</div>

<div>
    <h1 align="center" style="color:darkcyan;"><< One by One >></h1>
    <h1 align="center" style="color:darkcyan;">Coordinate with NearestNeighbors</h1>    
</div>

<div class="alert alert-success">  
</div>

#### - For this month's challenge, a different theme has been considered (Imputation). Thanks to Kaggle. But can we get help from "Ensembling" in this challenge as well?

#### - It is usually tedious to find the right coefficients for "Ensembling" and it is done with a lot of trial and error. also when you "Ensemble" between the results of two or more notebooks, you specify coefficients that are multiplied in all rows. In this way, the results will probably get better in some rows, and at the same time, in some other rows, the results may get worse. However, if the results are generally better, we consider this "Ensembling" successful.

#### - In any case, we should not forget that finding the coefficients for "Ensembling" with our eyes closed will probably make the results of some rows worse, but sometimes we do not realize this, because our overall score has improved anyway. And by the way, we all know that the shadow of "Overfitting" is always on "Ensembling" :)

#### - If the results submitted for a challenge are two-dimensional or multidimensional, it will certainly be more difficult to select the coefficients for the "ensemble" and perhaps only success with a lot of trial and error can be achieved. Of course, the results submitted for this challenge are one-dimensional (one column).

#### - In this notebook, we will share our innovative method for "Coordinate" the results and you will see that for each row, we perform separate calculations and determine the order of proximity of all points in a row to each other. Then we can use the point that has the highest score in this row as the main basis and, for example, combine the value of this point with the point closest to itself.

#### - We use "NearestNeighbors" for each row, which makes the calculations a bit slow. Of course, because the results of this challenge are one-dimensional (one column), you can use faster methods. But if the results of another challenge are [two-dimensional](https://www.kaggle.com/code/mehrankazeminia/gsdc22-coordinate-with-nearestneighbors) or multidimensional, "NearestNeighbors" is probably the best choice.

In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline
!ls ../input/*

<div class="alert alert-success">  
</div>

# <span style="color:darkcyan;">Data-Sets</span>

#### To illustrate this, We used the results of seven public notebooks. Thanks to everyone who published their great notebooks, especially:

##### @**shariful07**, @**cabaxiom**, @**abdulravoofshaik**, @**cv13j0**, @**ehekatlact**, @**alexryzhkov**, @**oxzplvifi**

In [ ]:
SAMPLE = pd.read_csv('../input/tabular-playground-series-jun-2022/sample_submission.csv')
display(SAMPLE)

In [ ]:
path0 = '../input/tps22jun87393/submission.csv' 
path1 = '../input/tps22jun87145/submission.csv'
path2 = '../input/tps22jun86869/submission.csv' 
path3 = '../input/tps22jun86868/nn_submission.csv' 
path4 = '../input/tps22jun86125/submission.csv'
path5 = '../input/tps22jun86042/nn_imputer.csv' 
path6 = '../input/tps22jun85974/submission.csv'

path  = [path0, path1, path2, path3, path4, path5, path6]

<div class="alert alert-success">  
</div>

# <span style="color:darkcyan;">Coordinate - One by One</span>

In [ ]:
Q = [[], [], [], [], [], [], []]
for k in range(len(path)):    
    sub_k = pd.read_csv(path[k]).values
    
    P = []
    for j in range(len(SAMPLE)):
        P.append([sub_k[j][1]])      
    Q[k] = P        

In [ ]:
def near_plt(points, best_score, support, generated):
    
    plt.style.use('seaborn-whitegrid') 
    plt.figure(figsize=(9, 9), facecolor='lightgray')
    plt.title(f'\nC O O R D I N A T E\n')   
    
    plt.scatter(points, points, s=120, facecolor='lightgray', edgecolor='black', label='All Points')
    plt.scatter(best_score, best_score, s=120, facecolor='violet', edgecolor='black', label='Best Score')
    plt.scatter(support, support, s=120, facecolor='yellow', edgecolor='black', label='Support')    
    plt.scatter(generated, generated, s=120, marker='x', label='Generated')
    
    plt.legend(fontsize=12, loc=2)
    plt.savefig('Coordinate_1.png')
    plt.show()

## **<span style="color:darkred;">Note:</span>**

#### You can compare the results of your notebook with many other notebooks. You can even use one or two or ... the nearest points. Then "Coordinate" each row of your results. Also, easily try different coefficients.

In [ ]:
from sklearn.neighbors import NearestNeighbors

V = [] # value
for i in range(len(SAMPLE)): 
    X = [Q[0][i], Q[1][i], Q[2][i], Q[3][i], Q[4][i], Q[5][i], Q[6][i]]
    
    nbrs = NearestNeighbors(n_neighbors=len(X), algorithm='ball_tree').fit(X)    
    _ , indices = nbrs.kneighbors(X)
 
    vv = (0.50 * X[indices[-1][0]][0]) + (0.50 * X[indices[-1][1]][0])
    V.append(vv) 
    
    if (i == 333111): # For Example
        print(f'\n\nFor Example(Row >> 333111)\n\n{SAMPLE.iloc[333111][:1]}')
        near_plt(X, [X[indices[-1][0]]] , [X[indices[-1][1]]] , [vv])

<div class="alert alert-success">  
</div>

In [ ]:
sub = SAMPLE.copy()
sub['value'] = V
sub

In [ ]:
sub.to_csv("submission.csv", index=False)
!ls

## <span style="color:darkred;">Coordinate >>> </span>
## <span style="color:darkred;">Public Score: 0.85787</span>

<div class="alert alert-success">  
</div>

<div class="alert alert-success">  
</div>

# <span style="color:darkcyan;">"Coordinate" Vs. "Ensembling"</span>

#### Please note that if we wanted to "Ensembling" the results of the two best notebooks (best score) together with our eyes closed, for example, the final result would not be better:

In [ ]:
s85974 = pd.read_csv(path6)
s86042 = pd.read_csv(path5)

In [ ]:
def gen_plt(main, support, generated):
    X  = main
    Y1 = support
    Y2 = generated
    
    plt.style.use('seaborn-whitegrid') 
    plt.figure(figsize=(9, 9), facecolor='lightgray')
    plt.title(f'\nE N S E M B L I N G\n')   
      
    plt.scatter(X, Y1, s=2.5, label='Support')    
    plt.scatter(X, Y2, s=2.5, label='Generated')
    plt.scatter(X, X , s=0.2, label='Main(X=Y)')
    
    plt.legend(fontsize=12, loc=2)
    plt.savefig('Ensembling_1.png')
    plt.show()

In [ ]:
ensemble = SAMPLE.copy()
ensemble['value'] = (s85974['value'] * 0.50) + (s86042['value'] * 0.50)
                                                   
gen_plt(s85974['value'], s86042['value'], ensemble['value'])

In [ ]:
ensemble.to_csv("ensembling.csv", index=False)
!ls

## <span style="color:darkred;">Ensembling >>></span>
## <span style="color:darkred;">Public Score: 1.17629</span>

<div class="alert alert-success">  
</div>

<div class="alert alert-success">
    <h1 align="center" style="color:darkcyan;">Good Luck.</h1> 
</div>